In [22]:
# Imoprts
import pandas as pd

## Step 1: Read the CSV File

In [23]:
# Create a file path
csv_file_path = "Data\CSV\jjones read_EPOCX_198673_2023.10.23T15.56.25.08.00.md.csv"

# Reading the CSV file
eeg_data = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
print(eeg_data.head())

C:\Users\Jason\AppData\Local\Temp\ipykernel_55348\501801612.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  eeg_data = pd.read_csv(csv_file_path)


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  title:jjones read  \
Timestamp         OriginalTimestamp EEG.Counter EEG.Interpolated EEG.AF3     EEG.F7      EEG.F3      EEG.FC5     EEG.T7      EEG.P7      EEG.O1      EEG.O2      EEG.P8      EEG.T8      EEG.FC6     EEG.F4      EEG.F8      EEG.AF4     EEG.RawCq   EEG.Battery EEG.BatteryPercent MarkerIndex MarkerType MarkerValueInt EEG.MarkerHardware CQ.AF3   CQ.F7    CQ.F3    CQ.FC5   

## Step 2: Preprocess the Data
Selecting Relevant Columns: remove extra information, select only the columns representing EEG data.
Timestamps and Sampling Rate: Ensure the data is correctly time-stamped and check the sampling rate if relevant for analysis.
Converting to Numeric: Convert it if necessary.

In [24]:
# Select only EEG data columns if necessary
# eeg_data = eeg_data[['column1', 'column2', ...]]

# Convert data to numeric
eeg_data = eeg_data.apply(pd.to_numeric, errors='coerce')


## Step 3: Artifact Removal and Filtering
For artifact removal and filtering, we need numerical data. Since pandas DataFrames are compatible with NumPy, we can use NumPy for mathematical operations. However, for more sophisticated EEG-specific processing, we may consider converting the DataFrame to a format compatible with MNE or other EEG processing libraries.

In [25]:
import numpy as np

# Example: Simple band-pass filter using NumPy (more complex filtering might require dedicated libraries)
# Define  filter parameters
low, high = 1, 40  # Example frequency range in Hz

# Apply the filter (this is a simplified example; real filtering is more complex)
eeg_filtered = np.where((eeg_data > low) & (eeg_data < high), eeg_data, 0)


## Step 5: Export Processed Data


In [27]:
# file path to save the processed data
output_csv_path = 'data/CSV/processed_csv_data.'
# Convert the NumPy array back to a pandas DataFrame
# Assuming 'eeg_filtered' is your NumPy array after filtering
eeg_filtered_df = pd.DataFrame(eeg_filtered, columns=eeg_data.columns)

# Now, export this DataFrame to CSV
eeg_filtered_df.to_csv(output_csv_path, index=False)
print(f"Processed data exported to {output_csv_path}")

Processed data exported to data/CSV/processed_csv_data.


In [28]:
# show the first few rows of the filtered data
print(eeg_filtered_df.head())

   title:jjones read   start timestamp:1698101785.435397  \
0                0.0                                 0.0   
1                0.0                                 0.0   
2                0.0                                 0.0   
3                0.0                                 0.0   
4                0.0                                 0.0   

    stop timestamp:1698103335.560894   headset type:EPOCX  \
0                                0.0                  0.0   
1                                0.0                  0.0   
2                                0.0                  0.0   
3                                0.0                  0.0   
4                                0.0                  0.0   

    headset serial:E2020A4F   headset firmware:720   channels:67  \
0                       0.0                    0.0           0.0   
1                       0.0                    0.0           0.0   
2                       0.0                    0.0           0.0   


In [29]:
# Check the first few rows of the data
print(eeg_data.head())

# Check basic statistics to understand the range and distribution
print(eeg_data.describe())


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               title:jjones read  \
Timestamp         OriginalTimestamp EEG.Counter EEG.Interpolated EEG.AF3     EEG.F7      EEG.F3      EEG.FC5     EEG.T7      EEG.P7      EEG.O1      EEG.O2      EEG.P8      EEG.T8      EEG.FC6     EEG.F4      EEG.F8      EEG.AF4     EEG.RawCq   EEG.Battery EEG.BatteryPercent MarkerIndex MarkerType MarkerValueInt EEG.MarkerHardware CQ.AF3   CQ.F7    CQ.F3    CQ.FC5   CQ.

## Review Filtering Logic
eeg_filtered = np.where((eeg_data > low) & (eeg_data < high), eeg_data, 0) would zero out all data not in the frequency range, which is not the correct approach for a band-pass filter. Instead, we want to zero out data that is outside the frequency range. The correct logic is eeg_filtered = np.where((eeg_data < low) | (eeg_data > high), 0, eeg_data).

In [30]:
# proper band-pass filtering using scipy (more complex filtering might require dedicated libraries)
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Assuming you know the sampling frequency (fs)
fs = 250  # Example: 250 Hz
lowcut = 1
highcut = 40

# Apply the filter to each EEG channel
eeg_filtered = eeg_data.apply(lambda x: bandpass_filter(x, lowcut, highcut, fs))


In [31]:
# Check the filtered data
print(eeg_filtered.head())
print(eeg_filtered.describe())


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               title:jjones read  \
Timestamp         OriginalTimestamp EEG.Counter EEG.Interpolated EEG.AF3     EEG.F7      EEG.F3      EEG.FC5     EEG.T7      EEG.P7      EEG.O1      EEG.O2      EEG.P8      EEG.T8      EEG.FC6     EEG.F4      EEG.F8      EEG.AF4     EEG.RawCq   EEG.Battery EEG.BatteryPercent MarkerIndex MarkerType MarkerValueInt EEG.MarkerHardware CQ.AF3   CQ.F7    CQ.F3    CQ.FC5   CQ.